Follow these steps to answer the question:

Identify the key information in the context.
Use the information to answer the question directly.
Provide evidence or reasoning from the context to support your answer.
Input:
Context: The water cycle describes how water moves through the environment. It involves processes like evaporation, condensation, precipitation, and collection. Evaporation occurs when water turns into vapor due to heat.
Question: What is evaporation, and how does it occur?

Output:

Key Information: Evaporation is a process in the water cycle where water turns into vapor.
Answer: Evaporation occurs when water turns into vapor due to heat.
Evidence: This process is part of the water cycle, as stated in the context.
Now, answer this question:

QAQC: 

General QAQC Review: Review the provided document for quality assurance. Identify any inconsistencies, factual inaccuracies, or areas requiring clarification. Suggest specific revisions to improve the document.

QAQC for Document Compliance and Policy Adherence: Assess whether the document complies with internal policies, industry standards, or legal requirements. Identify any areas of non-compliance and suggest corrective measures.

QAQC for Factual Accuracy and Completeness: Evaluate the text for factual accuracy and ensure all necessary details are included. Identify any gaps in information or potential errors, and propose solutions.

Grammar:

Professional Grammar Correction: Analyze the following text for grammatical accuracy, proper punctuation, and coherence. Provide a corrected version that adheres to formal English writing standards.

Advanced Sentence Refinement: Review the given sentence for grammatical errors, stylistic inconsistencies, and clarity. Rewrite it to ensure it meets the standards of formal professional communication.

Context-Specific Grammar and Style Improvement: Assess the provided text for grammatical accuracy and suitability for a formal professional setting. Provide an enhanced version with corrections and improved stylistic flow.

Q/A:

Analytical Question/Answer Based on Text: Text: "Regular exercise has been linked to improved mental health by reducing symptoms of anxiety and depression. Studies suggest that aerobic activities, such as running or swimming, are particularly effective."
Q: "Why might running or swimming be beneficial for mental health?"
A: "Running or swimming might be beneficial for mental health because these aerobic activities have been shown to reduce symptoms of anxiety and depression."

Inference-Based Question/Answer: Text: "After weeks of negotiations, the two countries signed a historic peace treaty, ending decades of conflict and opening the door to economic cooperation."
Q: "What might be a potential outcome of the peace treaty?"
A: "A potential outcome of the peace treaty could be increased economic cooperation between the two countries."

Comparative Question/Answer: Text: "City A has invested heavily in public transportation, resulting in reduced traffic congestion and lower carbon emissions. City B has focused on expanding highways, which has led to increased car usage and pollution."
Q: "How do the transportation strategies of City A and City B differ in their outcomes?"
A: "City A's strategy of investing in public transportation has reduced traffic congestion and carbon emissions, while City B's focus on highway expansion has increased car usage and pollution."

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def create_llama_prompt(system_prompt: str, user_prompt: str, eos_token: str="<|endoftext|>"):
    """
    Creates a formatted prompt to use with LLaMA models that include system-level and user-level prompts,
    with the specified EOS token for defining prompt boundaries.
    
    Parameters:
    system_prompt (str): Instructions or context provided to the model as the system-level prompt.
    user_prompt (str): The primary user query or command that requires a response.
    eos_token (str): The end-of-sequence token to mark the end of different sections of the prompt.
    
    Returns:
    str: A concatenated string that includes both system and user prompts, formatted with EOS tokens.
    """
    prompt = f"[SYSTEM]{eos_token} {system_prompt.strip()} {eos_token} [USER]{eos_token} {user_prompt.strip()} {eos_token}"
    return prompt

def generate_response(prompt: str, model_name: str="llama_model", max_new_tokens: int=200):
    """
    Generates a response using a LLaMA model with the given prompt.
    
    Parameters:
    prompt (str): The formatted prompt to pass to the LLaMA model.
    model_name (str): The name or path of the pre-trained model to load.
    max_new_tokens (int): The maximum number of tokens to generate for the response.
    
    Returns:
    str: The generated response from the LLaMA model.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

    # Tokenize the prompt and generate tokens
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)

    # Decode the output tokens and extract the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()  # Remove the prompt from the response

    return response

if __name__ == "__main__":
    # Example usage:
    system_prompt = "You are an assistant who answers questions helpfully and politely."
    user_prompt = "What is the difference between machine learning and deep learning?"
    
    # Create the formatted prompt
    formatted_prompt = create_llama_prompt(system_prompt, user_prompt)
    
    # Generate a response using a LLaMA model
    model_name = "decapoda-research/llama-7b-hf"  # Placeholder model path/name
    response = generate_response(formatted_prompt, model_name)
    
    # Print the generated response
    print("Response:")
    print(response)
